In [1]:
import fer

In [3]:
import node

In [5]:
import keras

In [6]:
import constant

In [8]:
import model

In [9]:
import utils

In [10]:
def test():
    import pandas
    pandas.read_csv(constant.train_data_csv_file_path)
    # pandas.read_csv(constant.train_data_csv_file_path)
test()

In [11]:
train_data_frame = fer.get_data_from_path(
    data_path=constant.train_data_csv_file_path,
    # data_path='../data_csv/train/data.csv',
    input_image_format='numerals',
    output_image_format='matrix',
    sampling_quantity=None,
)
train_inputs_as_array, train_labels_as_one_hot_encoded = model.keras_inputs_and_labels_from_data_frame( train_data_frame )

In [12]:
validate_data_frame = fer.get_data_from_path(
    data_path=constant.validate_data_csv_file_path,
    input_image_format='numerals',
    output_image_format='matrix',
    sampling_quantity=None,
)
validate_inputs_as_array, validate_labels_as_one_hot_encoded = model.keras_inputs_and_labels_from_data_frame( validate_data_frame )

In [13]:
test_data_frame = fer.get_data_from_path(
    data_path=constant.test_data_csv_file_path,
    input_image_format='numerals',
    output_image_format='matrix',
    sampling_quantity=None,
)
test_inputs_as_array, test_labels_as_one_hot_encoded = model.keras_inputs_and_labels_from_data_frame( test_data_frame )

In [14]:
# train_inputs_as_array_and_train_labels_as_one_hot_encoded = keras.utils.image_dataset_from_directory(
#     directory='../data_png/train/',
#     labels='inferred',
#     label_mode='categorical',
#     color_mode='grayscale',
#     batch_size=32,
#     image_size=(48,48),
# )
# # dataset genreator of (32*input, 32*label)
# # train_labels_as_one_hot_encoded = None

In [15]:
# validate_inputs_as_array_and_validate_labels_as_one_hot_encoded = keras.utils.image_dataset_from_directory(
#     directory='../data_png/validate/',
#     labels='inferred',
#     label_mode='categorical',
#     color_mode='grayscale',
#     batch_size=32,
#     image_size=(48,48),
# )
# # dataset genreator of (32*input, 32*label)
# validate_labels_as_one_hot_encoded = None

In [16]:
# test_inputs_as_array_and_test_labels_as_one_hot_encoded = keras.utils.image_dataset_from_directory(
#     directory='../data_png/test/',
#     labels='inferred',
#     label_mode='categorical',
#     color_mode='grayscale',
#     batch_size=32,
#     image_size=(48,48),
# )
# # dataset genreator of (32*input, 32*label)
# test_labels_as_one_hot_encoded = None

In [17]:
def compiled_model_from_datas_graph(
    data_graph,
    input_shape=(48,48),
):
    last_data_node = data_graph[-1]
    first_data_node = data_graph[0]
    keras_model = node.path_keras_model_from_last_data_node(
        last_data_node=last_data_node,
        input_shape=input_shape,
        first_data_node=first_data_node,
    )
    keras_model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'],
    )
    keras_model.summary()
    return keras_model


def trained_model_from_compiled_model(
    compiled_model,
    train_inputs_as_array,
    train_labels_as_one_hot_encoded,
    validate_inputs_as_array,
    validate_labels_as_one_hot_encoded,
    epochs=2,
    verbose=1,
    batch_size=16,
):
    import time 
    class TimeOut(keras.callbacks.Callback):
        def __init__(self, t0, timeout):
            super().__init__()
            self.t0 = t0
            self.timeout = timeout  # time in minutes
        def on_train_batch_end(self, batch, logs=None):
            if time.time() - self.t0 > self.timeout * 60:  # 58 minutes
                print(f"\nReached {(time.time() - self.t0) / 60:.3f} minutes of training, stopping")
                self.model.stop_training = True

    history = compiled_model.fit(
        x=train_inputs_as_array,
        y=train_labels_as_one_hot_encoded,
        validation_data=(validate_inputs_as_array,validate_labels_as_one_hot_encoded),
        epochs=epochs,
        batch_size=batch_size,
        validation_batch_size=batch_size,
        callbacks=[
                keras.callbacks.EarlyStopping(patience=3, restore_best_weights=False, monitor='accuracy', min_delta=0.005, verbose=1),
                keras.callbacks.EarlyStopping(patience=4, restore_best_weights=False, monitor='val_accuracy', min_delta=0.005, verbose=1),
                TimeOut(t0=time.time(), timeout=120), # put 1 for testing
            ],
        verbose=verbose,
    )
    return compiled_model


def test_accuracy_from_model_and_test_set(
    trained_model, 
    test_inputs_as_array,
    test_labels_as_one_hot_encoded,
    batch_size=16,
):
    test_loss, test_accuracy = trained_model.evaluate(test_inputs_as_array, test_labels_as_one_hot_encoded, batch_size=batch_size)
    print(f"output of evaluate method: {test_loss = }, {test_accuracy =} ")
    return test_accuracy


In [18]:
def saved_trained_model_file_path_from_data_graph(
    data_graph,
    train_inputs_as_array,
    train_labels_as_one_hot_encoded,
    validate_inputs_as_array,
    validate_labels_as_one_hot_encoded,
    test_inputs_as_array,
    test_labels_as_one_hot_encoded,
    destination_folder_path=constant.model_folder_path,
    file_name='badly_trained_reference_model',
    epochs=2, # for testing
    # epochs=13,# aim for 40 to reach 60%
):
    compiled_model = compiled_model_from_datas_graph( data_graph, input_shape=(48,48), )
    trained_model = trained_model_from_compiled_model(
        compiled_model,
        train_inputs_as_array,
        train_labels_as_one_hot_encoded,
        validate_inputs_as_array,
        validate_labels_as_one_hot_encoded,
        epochs=epochs,
    )
    print(
        test_accuracy_from_model_and_test_set(
            trained_model,
            test_inputs_as_array,
            test_labels_as_one_hot_encoded,
        )
    )
    keras.saving.save_model(
        model=trained_model, 
        filepath=constant.emotion_recognition_model_file_path, 
        overwrite=True, 
        zipped=None,
    )
    return constant.emotion_recognition_model_file_path

In [19]:
def test():    
    architecture_as_datas = utils.object_from_pickle_file_path(constant.architecture_file_path)
    architecture_as_data_graph = node.path_data_graph_from_datas( architecture_as_datas )
    return saved_trained_model_file_path_from_data_graph(
        data_graph= architecture_as_data_graph ,
        train_inputs_as_array=train_inputs_as_array,
        train_labels_as_one_hot_encoded=train_labels_as_one_hot_encoded,
        validate_inputs_as_array=validate_inputs_as_array,
        validate_labels_as_one_hot_encoded=validate_labels_as_one_hot_encoded,
        test_inputs_as_array=test_inputs_as_array,
        test_labels_as_one_hot_encoded=test_labels_as_one_hot_encoded,
        destination_folder_path=constant.model_folder_path,
        file_name=constant.emotion_recognition_model_file_name,
    )
saved_model_file_path = test()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 48, 48)         │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation              │ (None, 48, 48)         │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 48, 48)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 44, 44, 64)     │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 44, 44, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 40, 40, 64)     │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 40, 40, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 20, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 18, 18, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 6, 6, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 6, 6, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 4, 4, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │             

 Total params: 1,347,015 (5.14 MB)

 Trainable params: 1,344,967 (5.13 MB)

 Non-trainable params: 2,048 (8.00 KB)

Epoch 1/2
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 175s 96ms/step - accuracy: 0.2017 - loss: 2.3954 - val_accuracy: 0.2246 - val_loss: 1.8482
Epoch 2/2
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 273s 152ms/step - accuracy: 0.3206 - loss: 1.7015 - val_accuracy: 0.3338 - val_loss: 1.7076
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.3307 - loss: 1.7152
output of evaluate method: test_loss = 1.7200125455856323, test_accuracy =0.3265533447265625 
0.3265533447265625


ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=../model/emotion_recognition.

In [ ]:
saved_model_file_path

In [ ]:
utils.object_from_file_path(saved_model_file_path).summary()